<a href="https://colab.research.google.com/github/jglaser/gigadocking_notebooks/blob/main/extract_top.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [6]:
project_id = 'cmb-covid19-discovery-collab'

In [7]:
%%time
%%bigquery --project $project_id df
SELECT scores.name, score, smiles.smiles FROM mpro_5r84.confs_all AS confs
INNER JOIN mpro_5r84.scores_rank0 AS scores ON scores.name = confs.name
INNER JOIN vf_2018.smiles ON smiles.name = confs.name
WHERE score IS NOT NULL and rank = 19 # 0
ORDER BY score 
LIMIT 10000

CPU times: user 156 ms, sys: 6.5 ms, total: 163 ms
Wall time: 1.8 s


In [8]:
df

,name,score,smiles
0,Z71169259_1_T1,-12.06,O=C(NC[C@@H]1CC2c3ccccc3C1c1ccccc21)c1ccc2c(c1...
1,Z2150409399_1_T1,-11.62,O=C(Nc1cccc(c1)C#Cc1cccs1)N1CCC2(C1)Oc1ccccc1O2
2,Z2910540677_24_T1,-11.58,CC1CN(CCN(C1)C(=O)[C@H]1[C@H]2CCC(=O)[C@H]12)C...
3,Z1283902295_2_T1,-11.52,CC1(C)[C@@H](NC(=O)N2Cc3ccccc3-c3ccccc3C2)[C@@...
4,Z2079388592_1_T1,-11.43,O=C(N1CCC2(CCOCC2)CC1)c1cc(nc2ccccc12)-c1ccc2O...
...,...,...,...
9995,Z1217403322_1_T1,-9.87,O=C(N1CCc2ccccc2CC1)c1ccc2CCCc2c1
9996,PV-001874494732_1_T1,-9.87,Cc1ccc(o1)C(=O)Nc1c([nH]c2ccccc12)-c1ccccc1
9997,Z2180413728_2_T1,-9.87,O=C(NCC1Cc2ccccc2C1)N[C@H]1c2ccccc2CC11CCOCC1
9998,Z1172066310_1_T2,-9.87,O=C(N1CCC[C@@H]1C1=CC=CC2=CC=CC=C12)C1=CC=C2NN...


In [ ]:
df.to_parquet('smiles_adgpu_top10k.parquet')

In [ ]:
from google.colab import files
files.download('smiles_adgpu_top10k.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# also extract conformation
%%time
%%bigquery --project $project_id df
SELECT confs.name, confs.rank, confs.conf, confs.score, __residual_activity  FROM mpro_5r84.confs_all AS confs
INNER JOIN mpro_exp.hit_expansion AS hits ON STARTS_WITH(confs.name, hits.Inhibitor)

CPU times: user 7.19 s, sys: 743 ms, total: 7.93 s
Wall time: 6min 10s


In [11]:
df

,name,rank,conf,score,__residual_activity
0,Z1033130370_1_T1,0,MODEL 4\nUSER Run = 4\nUSER\nUSER ...,-6.94,0.52
1,Z1033130370_1_T1,1,MODEL 12\nUSER Run = 12\nUSER\nUSER ...,-6.78,0.52
2,Z1033130370_1_T1,2,MODEL 2\nUSER Run = 2\nUSER\nUSER ...,-6.64,0.52
3,Z1033130370_1_T1,3,MODEL 9\nUSER Run = 9\nUSER\nUSER ...,-6.58,0.52
4,Z1033130370_1_T1,4,MODEL 8\nUSER Run = 8\nUSER\nUSER ...,-6.53,0.52
...,...,...,...,...,...
5555,Z2434300525_1_T1,15,MODEL 5\nUSER Run = 5\nUSER\nUSER ...,-7.64,0.48
5556,Z2434300525_1_T1,16,MODEL 20\nUSER Run = 20\nUSER\nUSER ...,-7.49,0.48
5557,Z2434300525_1_T1,17,MODEL 11\nUSER Run = 11\nUSER\nUSER ...,-7.46,0.48
5558,Z2434300525_1_T1,18,MODEL 17\nUSER Run = 17\nUSER\nUSER ...,-7.06,0.48


In [24]:
pdb_dir = 'map4'
def write_conf(name_rank_conf):
  name, rank, conf = name_rank_conf
  with open(pdb_dir+'/'+name+'-'+str(rank)+'.pdbqt','w') as f:
    f.write(conf)

In [25]:
!mkdir -p $pdb_dir
df[['name','rank','conf']].apply(write_conf,axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5555    None
5556    None
5557    None
5558    None
5559    None
Length: 5560, dtype: object

In [26]:
! zip -r map4.zip map4

Streaming output truncated to the last 5000 lines.
  adding: map4/Z1692806105_1_T1-0.pdbqt (deflated 66%)
  adding: map4/Z1888670756_2_T1-0.pdbqt (deflated 68%)
  adding: map4/Z2702030288_1_T1-12.pdbqt (deflated 67%)
  adding: map4/Z1580833935_3_T1-9.pdbqt (deflated 66%)
  adding: map4/Z1530725178_1-3.pdbqt (deflated 66%)
  adding: map4/Z2254357454_1_T1-6.pdbqt (deflated 66%)
  adding: map4/Z1530789835_2_T1-13.pdbqt (deflated 65%)
  adding: map4/Z2100359935_4_T1-7.pdbqt (deflated 66%)
  adding: map4/Z1528050012_2_T1-4.pdbqt (deflated 66%)
  adding: map4/Z1843343123_3_T1-14.pdbqt (deflated 66%)
  adding: map4/Z1692806105_2_T1-4.pdbqt (deflated 66%)
  adding: map4/Z18886707_1_T1-5.pdbqt (deflated 65%)
  adding: map4/Z2044131801_2_T1-19.pdbqt (deflated 66%)
  adding: map4/Z1528050012_1_T1-1.pdbqt (deflated 66%)
  adding: map4/Z1577380181_7_T1-17.pdbqt (deflated 66%)
  adding: map4/Z1888670741_2_T1-13.pdbqt (deflated 68%)
  adding: map4/Z1888670700_1_T1-5.pdbqt (deflated 65%)
  adding: map

In [28]:
df[['name','rank','score','__residual_activity']].to_csv('map4.csv')

In [30]:
from google.colab import files
files.download('map4.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#files.download('map4.csv')